**AutoML OSS入門（2）**

# 4行でモデル構築と予測ができるAutoML OSSの老舗「auto-sklearn」

本ノートブックの紹介記事と併せてご覧ください。
- ＠IT連載 AutoML OSS入門（２）- 第2回「4行でモデル構築と予測ができるAutoML OSSの老舗「auto-sklearn」を解説」

また、ノートブックの扱い方やタイタニックの生存予測データについては、連載記事の第1回を参照してください。
- [＠IT連載 AutoML OSS入門（１）- 第1回「機械学習モデル構築作業の煩雑さを解消する「AutoML」とは――歴史、動向、利用のメリットを整理する」](https://www.atmarkit.co.jp/ait/articles/2107/02/news006.html)

## タイタニックデータでAutoML
タイタニックの生存予測データを使って、auto-sklearnを紹介していきます。



### セットアップ
auto-sklearnをインストールします。


In [1]:
# 依存するパッケージのインストール
# 注) 記事執筆後にColab環境が少し変わったのでimgaugのインストール追加
!pip install imgaug==0.2.6

!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install

# auto-sklearnのインストール
!pip install auto-sklearn

     |████████████████████████████████| 631 kB 6.8 MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654020 sha256=7d0e64b6a11de466ae158c8a4e836e1a81bd1ad8c2730a6ed6f1695547d6257d
  Stored in directory: /root/.cache/pip/wheels/89/72/98/3ebfdba1069a9a8eaaa7ae7265cfd67d63ef0197aaee2e5f9c
Successfully built imgaug
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   249  100   249    0     0   1317      0 --:--:-- --:--:-- --:--:--  1317
     |████████████████████████████████| 28.5 MB 52 kB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
     |████████████████████████████████| 22.3 MB 1.5 MB/s 
  Attem

(注) ランタイムを再起動してください。
- ランタイムの再起動方法<br>
ランタイム > ランタイムを再起動 を実行

In [1]:
# auto-sklearnのバージョンを表示してインストールを確認
# バージョンが表示されればOK!!
# 注) 記事執筆後にauto-sklearnのバージョンが0.13.0にup
import autosklearn
print(autosklearn.__version__)

0.13.0


### データのロード
タイタニックの生存予測データをダウンロードし、データを読み込みます。<br>
第1回記事の「タイタニックの生存予測データの取得方法と解説」を参照してください。

In [2]:
# データの準備
!wget -N https://github.com/aiq2020-tw/automl-notebooks/raw/main/titanic.zip
!unzip titanic.zip

--2021-08-25 14:35:38--  https://github.com/aiq2020-tw/automl-notebooks/raw/main/titanic.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aiq2020-tw/automl-notebooks/main/titanic.zip [following]
--2021-08-25 14:35:38--  https://raw.githubusercontent.com/aiq2020-tw/automl-notebooks/main/titanic.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34877 (34K) [application/zip]
Saving to: ‘titanic.zip’

titanic.zip         100%[===================>]  34.06K  --.-KB/s    in 0.001s  

Last-modified header missing -- time-stamps turned off.
2021-08-25 14:35:38 (36.4 MB/s) - ‘titanic.zip’ saved [3487

In [3]:
# 学習データ/テストデータの読み込み
import pandas as pd

train_df = pd.read_csv('train.csv')
print('Train data shape:', str(train_df.shape))
test_df = pd.read_csv('test.csv')
print('Test data shape:', str(test_df.shape))

Train data shape: (891, 12)
Test data shape: (418, 11)


### データの前処理
データを説明変数と目的変数に分けます。<br>
また、カテゴリカル変数は数値変数に変換します。

In [4]:
# 目的変数
y_train = train_df.pop('Survived')
# 説明変数（「PassengerId」は除去）
X_train = train_df.drop(['PassengerId'], axis=1)
X_test = test_df.drop('PassengerId', axis=1)

In [5]:
# データタイプ確認
X_train.dtypes

Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object

In [6]:
# カテゴリカル変数を数値変数に変換
# 注) 本来カテゴリカル変数にはタイプとして「category」を設定すべきだが、今回は未設定。
from sklearn.preprocessing import LabelEncoder

list_cols = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
for col in list_cols:
    target_column = pd.concat([X_train[col], X_test[col]])
    le = LabelEncoder()
    le.fit(target_column)
    X_train[col] = le.transform(X_train[col])
    # X_train[col] = X_train[col].astype('category')
    X_test[col] = le.transform(X_test[col])
    # X_test[col] = X_test[col].astype('category')
X_train.dtypes

Pclass        int64
Name          int64
Sex           int64
Age         float64
SibSp         int64
Parch         int64
Ticket        int64
Fare        float64
Cabin         int64
Embarked      int64
dtype: object

### モデルの学習
`AutoSklearnClassifier`クラスを使ってモデルを学習します。

In [7]:
from autosklearn.classification import AutoSklearnClassifier

# モデルのインスタンス生成
cls = AutoSklearnClassifier(time_left_for_this_task=120, seed=42)
# モデルの学習
cls.fit(X_train, y_train)

AutoSklearnClassifier(per_run_time_limit=12, seed=42,
                      time_left_for_this_task=120)

In [8]:
# 学習済みモデルの情報
print(cls.sprint_statistics())
print(cls.show_models())

auto-sklearn results:
  Dataset name: fe3cdf50-05b1-11ec-81c5-0242ac1c0002
  Metric: accuracy
  Best validation score: 0.833898
  Number of target algorithm runs: 34
  Number of successful target algorithm runs: 33
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

[(0.380000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'adaboost', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'robust_scaler', 'feature_preprocessor:__choice__': 'extra_trees_preproc_for_classification', 'classifier:adaboost:algorithm': 'SAMME.R', 'classifier:adaboost:learning_r

### 予測
テストデータを予測して、Kaggleへの提出ファイルを作成します。

In [9]:
# 予測
test_df['Survived'] = cls.predict(X_test)
# csvファイル作成
test_df[['PassengerId','Survived']].to_csv('submission.csv', index=None)

## その他の機能と応用
可視化ツールのPipelineProfilerとauto-sklearn 2.0を紹介します。

### PipelineProfiler
PipelineProfilerツールを使用して、学習済みモデルの内部情報を可視化してみましょう。

In [10]:
!pip install pipelineprofiler

     |████████████████████████████████| 880 kB 8.4 MB/s 


In [11]:
import PipelineProfiler
profiler_data = PipelineProfiler.import_autosklearn(cls)
PipelineProfiler.plot_pipeline_matrix(profiler_data)

Output hidden; open in https://colab.research.google.com to view.

出力結果をマウス操作することで、学習済みモデルの内部情報を確認することができます。<br>
いくつか機能を紹介します。
- パイプライン(例として auto-sklearn #1)上のプリミティブタイプにマウスオーバーすると、その処理のタイプやハイパーパラメーターなどの情報を確認できる。
- 上部の前処理や分類器等をクリックすると、ハイパーパラメーターとその値が表示される。
- パイプラインをマウスで選択すると、パイプラインの構造が表示される。
- 「ACCURACY」をクリックして「ENSEMBLE WEIGHT」などを選択すると、選択された条件でパイプラインを並び替えられる。

### auto-sklearn 2.0
auto-sklearn 2.0で上記のauto-sklearnと同様にモデルを学習し、予測を行ってみましょう。使い方は同じです。

In [12]:
from autosklearn.experimental.askl2 import AutoSklearn2Classifier

# 学習
cls2 = AutoSklearn2Classifier(time_left_for_this_task=120, seed=42)
cls2.fit(X_train, y_train)
# 予測
test_df['V2_Survived'] = cls2.predict(X_test)

/usr/local/lib/python3.7/dist-packages/smac/intensification/parallel_scheduling.py:155: UserWarning: SuccessiveHalving is intended to be used with more than 1 worker but num_workers=1
  num_workers


以上で、タイタニックの生存予測データを使ったauto-sklearnの紹介は終わりです。